In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow.keras.layers import ReLU, concatenate
path='/content/drive/MyDrive/assignment 1/annotations.csv'
relative_path='/content/drive/MyDrive/assignment 1/train/train/'
from tensorflow.keras.optimizers import SGD

In [ ]:
def load_data(path,relative_path):
    df=pd.read_csv(path)
    data=[]
    for filename in df['fname']:
        address=relative_path+filename
        data.append(np.load(address))
    label=df['label']
    return data,label


In [ ]:
data,label=load_data(path,relative_path)


In [ ]:
mx_len=0
mn_len=100000
tot=0
for spectrograms in data:
    mx_len=max(mx_len,spectrograms.shape[2])
    mn_len=min(mn_len,spectrograms.shape[2])
    tot+=spectrograms.shape[2]
tot/1000,mn_len,mx_len

(593.504, 29, 2584)

In [ ]:
def list_to_np(data,max_len,label):
    final_data=[]
    labels=[]
    for spectrogram,l in zip(data,label):
        x=spectrogram.flatten()
        k=len(x)/128
        if k>600:
          k=(int)(k/600)+1
          for i in range (k):
            if i!=k-1:
              final_data.append(x[i*600*128:(i+1)*600*128])
              labels.append(l)
            else:
              y=np.zeros((600*k*128-len(x)))
              y=np.append(x[(k-1)*600*128:],y)
              final_data.append(y)
              labels.append(l)
        else:
            y=np.zeros((600*128-len(x)))
            y=np.append(x,y)
            final_data.append(y)
            labels.append(l)
    return np.array(final_data),labels

In [ ]:

data,label=list_to_np(data,mx_len,label)

In [ ]:
label=pd.get_dummies(label)

In [ ]:


def d1(data):
  d=[]
  for i in range(len(data)):
    k=data[i].flatten()
    k=np.reshape(k,(128,600,1))
  
    d.append(k)
  d=np.array(d)
  return d
d=d1(data)
train_x,test_x,train_y,test_y=train_test_split(d,label,test_size=0.2,random_state=711)
del d

In [ ]:
model = Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=(128,600,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
opt = SGD(learning_rate=0.01, momentum=0.9)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
 history = model.fit(train_x, train_y, epochs=60,batch_size=32)

Epoch 1/100
40/40 [==============================] - 13s 93ms/step - loss: 2.5438 - accuracy: 0.1500
Epoch 2/100
40/40 [==============================] - 4s 93ms/step - loss: 2.1203 - accuracy: 0.2555
Epoch 3/100
40/40 [==============================] - 4s 93ms/step - loss: 1.8328 - accuracy: 0.3602
Epoch 4/100
40/40 [==============================] - 4s 94ms/step - loss: 1.2990 - accuracy: 0.5891
Epoch 5/100
40/40 [==============================] - 4s 94ms/step - loss: 1.0824 - accuracy: 0.6781
Epoch 6/100
40/40 [==============================] - 4s 94ms/step - loss: 1.0865 - accuracy: 0.7117
Epoch 7/100
40/40 [==============================] - 4s 94ms/step - loss: 0.8792 - accuracy: 0.7516
Epoch 8/100
40/40 [==============================] - 4s 94ms/step - loss: 0.7635 - accuracy: 0.7812
Epoch 9/100
40/40 [==============================] - 4s 94ms/step - loss: 1.2725 - accuracy: 0.6906
Epoch 10/100
40/40 [==============================] - 4s 94ms/step - loss: 1.5706 - accuracy: 0.571

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(test_x,test_y,verbose=2)

11/11 - 1s - loss: 5.8552 - accuracy: 0.5857 - 663ms/epoch - 60ms/step


[5.855224132537842, 0.585669755935669]